In [ ]:
import re
from collections import namedtuple, OrderedDict
from datetime import datetime
from os import listdir
from os.path import join as join_path, exists as file_exists, realpath, expanduser, basename, splitext

import numpy as np
import pandas as pd
from bokeh.models import HoverTool, CDSView, BooleanFilter
from bokeh.plotting import figure, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.resources import INLINE
from bokeh.io import output_notebook, show
from bokeh.palettes import Spectral6
from bokeh.models import NumeralTickFormatter
from bokeh.transform import factor_cmap


from classifiers import bucket_alist
from colors import from_file as color_from_file
from images import from_file as image_from_file, NeuralNetwork
from trial_dataframe import create_tribulation, create_color_tribulation, create_image_tribulation

output_notebook(resources=INLINE)

## Functions

In [ ]:
def filter_improvements(df):
    new_df = pd.DataFrame()
    for int_labels in df['int_labels'].unique():
        label_df = df[df['int_labels'] == int_labels]
        improvement = label_df[['num_epochs', 'true_positive_init']].drop_duplicates().sort_values('num_epochs')
        epochs = set([improvement.iloc[0]['num_epochs']])
        last_accuracy = 0
        for row in improvement.itertuples():
            if row.true_positive_init > last_accuracy:
                epochs.add(row.num_epochs)
                last_accuracy = row.true_positive_init
        new_df = pd.concat([new_df, df[(df['int_labels'] == int_labels) & (df['num_epochs'].isin(epochs))]])
    return new_df

def plot_tribulation_roc(source):
    roc_fig = figure(
        width=400, height=400,
        x_axis_label='Proportion of All Data Searched',
        y_axis_label='Proportion of Label Data Found',
        tools=['box_select'],
    )
    roc_fig.line(
        x='searched',
        y='found',
        selection_color='red',
        source=source,
    )
    return roc_fig

def plot_tribulation_regret(source):
    if not isinstance(source, ColumnDataSource):
        source = ColumnDataSource(source)
    regret_fig = figure(
        width=400, height=400,
        x_range=[0, 1.1],
        x_axis_label='Classifier Test Accuracy',
        y_range=[0, 1.1],
        y_axis_label='Mean Regret (lower is better)',
        tools=['box_select'],
    )
    renderer = regret_fig.x(
        x='true_positive_init',
        y='mean_regret_scaled',
        selection_color='red',
        source=source,
    )
    regret_fig.add_tools(HoverTool(renderers=[renderer], tooltips=[
        ('trial', '@trial_id'),
        ('accuracy', '@true_positive_init'),
        ('regret', '@mean_regret_scaled'),
    ]))
    return regret_fig

def plot_tribulation(df):
    source = ColumnDataSource(df)
    roc_fig = plot_tribulation_roc(source)
    regret_fig = plot_tribulation_regret(source)
    figures = [roc_fig, regret_fig]
    return gridplot([figures])

## Demo Data

In [ ]:
def regret_demo():
    
    def create_plot_data(df, y):
        return pd.DataFrame({
            'index': [0],
            f'{y}_percent': [0],
            f'{y}_sum': [0],
        }).append(df.sort_values(f'{y}_rank'), sort=False)
        
    df = pd.DataFrame({
        'index': list(range(1, 6)),
        'misclass': [180, 20, 5, 2, 1],
        'heuristic': [1, 6, 3, 4, 4],
    })
    df = df.sort_values('misclass', ascending=False)
    df['misclass_rank'] = list(range(1, 6))
    df['misclass_percent'] = [n / 5 for n in range(1, 6)]
    df['misclass_sum'] = df['misclass'].cumsum() / df['misclass'].sum()
    df = df.sort_values('heuristic')
    df['heuristic_rank'] = list(range(1, 6))
    df['heuristic_percent'] = [n / 5 for n in range(1, 6)]
    df['heuristic_sum'] = df['misclass'].cumsum() / df['misclass'].sum()
    display(df)
    
    fig = figure(
        width=1200, height=800,
        x_range=[0, 1.05],
        y_range=[0, 1.05],
        x_axis_label='% Searched',
        y_axis_label='% Found',
    )
    renderers = []
    # shading
    fig.patch(
        x=[0.2, 0.4, 0.6, 0.8, 1.0, 0.8, 0.6, 0.4, 0.2],
        y=[0.865385, 0.961538, 0.985577, 0.995192, 1.000000, 0.903846, 0.899038, 0.889423, 0.865385],
        color=Spectral6[2],
        #hatch_pattern='/', # only in Bokeh > 1.1.0
    )
    # misclass
    renderers.append(fig.line(
        x=f'misclass_percent',
        y=f'misclass_sum',
        source=ColumnDataSource(create_plot_data(df, 'misclass')),
        legend='Ground-truth',
        line_color=Spectral6[1],
        line_width=7,
        line_dash='dashed',
    ))
    # heuristic
    renderers.append(fig.line(
        x=f'heuristic_percent',
        y=f'heuristic_sum',
        source=ColumnDataSource(create_plot_data(df, 'heuristic')),
        legend='Heuristic',
        line_color=Spectral6[0],
        line_width=7,
    ))
    # random
    renderers.append(fig.line(
        x=[0, 1],
        y=[0, 1],
        legend='Random',
        line_color=Spectral6[5],
        line_width=7,
        line_dash='dotted',
    ))
        
    fig.add_tools(HoverTool(renderers=renderers, tooltips=[
        ('index', '@index'),
        ('misclass', '@misclass'),
        ('heuristic', '@heuristic'),
    ]))
    fig.outline_line_color = None
    fig.axis.axis_line_width = 5
    fig.axis.axis_label_text_font_size = '30pt'
    fig.axis.major_label_text_font_size = '30pt'
    fig.grid.grid_line_color = '#CCCCCC'
    fig.axis.minor_tick_in = 0
    fig.axis.minor_tick_out = 0
    fig.legend.location = 'bottom_right'
    fig.legend.label_text_font_size = '30pt'
    fig.legend.glyph_width = 100
    fig.xaxis[0].formatter = NumeralTickFormatter(format='0%')
    fig.yaxis[0].formatter = NumeralTickFormatter(format='0%')
    show(fig)
regret_demo()

## Color Domain

### Regret vs. Data Size

In [ ]:
def create_color_plot(num_centroids, num_new_labels, plot_df, fig=None):
    if fig is None:
        fig = figure(
            width=200, height=200,
            title='{} + {} = {}'.format(num_centroids, num_new_labels - num_centroids, num_new_labels),
            x_axis_type='log',
            x_range=[100, 1100000],
            y_range=[0, 0.025],
            x_axis_label='Dataset Size',
            y_axis_label='Mean Regret (lower is better)',
        )
    fig.line(
        x='dataset_size',
        y='mean_regret_scaled',
        source=ColumnDataSource(plot_df[
            (plot_df['num_centroids'] == num_centroids) & (plot_df['num_colors'] == num_new_labels)
        ]),
    )
    return fig

def plot_color_regret():
    # load data
    df = create_color_tribulation('colors')
    # group by independent variables
    plot_df = df[[
        'trial_id',
        'random_seed', 'num_centroids', 'dataset_size', 'num_colors',
        'mean_regret_scaled',
    ]].drop_duplicates().groupby([
        'num_centroids', 'dataset_size', 'num_colors'
    ]).mean()['mean_regret_scaled'].reset_index()

    plot_df['num_centroids'] = plot_df['num_centroids'].astype(int)
    plot_df['dataset_size'] = plot_df['dataset_size'].astype(int)
    plot_df['num_colors'] = plot_df['num_colors'].astype(int)
    plot_df = plot_df[['num_centroids', 'dataset_size', 'num_colors', 'mean_regret_scaled']].drop_duplicates()
    # build grid plot
    grid = []
    for r, num_centroids in enumerate(reversed([10, 20, 50, 100])):
        row = []
        for c, num_new_labels in enumerate([20, 50, 100, 200]):
            if num_new_labels <= num_centroids:
                row.append(None)
                continue
            fig = create_color_plot(num_centroids, num_new_labels, plot_df)
            row.append(fig)
        grid.append(row)
    return gridplot(grid)

show(plot_color_regret())

In [ ]:
def create_color_plot(num_old_labels, total_labels, plot_df, fig):
    line_dash = {
        190: 'solid',
        # for scaling old labels
        180: 'dashed',
        150: 'dotdash',
        100: 'dotted',
        # for scaling new labels
        90: 'dashed',
        40: 'dotdash',
        10: 'dotted',
    }
    fig.line(
        x='dataset_size',
        y='mean_regret_scaled',
        source=ColumnDataSource(plot_df[
            (plot_df['num_centroids'] == num_old_labels) & (plot_df['num_colors'] == total_labels)
        ]),
        line_width=5,
        legend=f'{num_old_labels} old labels, {total_labels - num_old_labels} new labels',
        line_dash=line_dash[total_labels - num_old_labels],
    )
    return fig

def create_color_scaling_plot():
    # load data
    df = create_color_tribulation('colors')
    # group by independent variables
    plot_df = df[[
        'trial_id',
        'random_seed', 'num_centroids', 'dataset_size', 'num_colors',
        'mean_regret_scaled',
    ]].drop_duplicates().groupby([
        'num_centroids', 'dataset_size', 'num_colors'
    ]).mean()['mean_regret_scaled'].reset_index()

    plot_df['num_centroids'] = plot_df['num_centroids'].astype(int)
    plot_df['dataset_size'] = plot_df['dataset_size'].astype(int)
    plot_df['num_colors'] = plot_df['num_colors'].astype(int)
    plot_df = plot_df[['num_centroids', 'dataset_size', 'num_colors', 'mean_regret_scaled']].drop_duplicates()
    
    fig = figure(
        width=1200, height=800,
        #title='Scaling Old Labels',
        x_axis_type='log',
        x_range=[750, 1100000],
        y_range=[0, 0.0275],
        x_axis_label='Dataset Size',
        y_axis_label='Mean Regret (lower is better)',
    )
    total_labels = 200
    for num_old_labels in [10, 20, 50, 100]:
        if total_labels <= num_old_labels:
            continue
        fig = create_color_plot(num_old_labels, total_labels, plot_df, fig)
    fig.outline_line_color = None
    fig.axis.axis_line_width = 5
    fig.axis.axis_label_text_font_size = '30pt'
    fig.axis.major_label_text_font_size = '30pt'
    fig.grid.grid_line_color = None
    fig.axis.minor_tick_in = 0
    fig.axis.minor_tick_out = 0
    fig.legend.location = 'top_right'
    fig.legend.label_text_font_size = '30pt'
    fig.legend.glyph_width = 100
    fig.xaxis[0].formatter = NumeralTickFormatter(format="0a")
    show(fig)
    
    fig = figure(
        width=1200, height=800,
        #title='Scaling New Labels',
        x_axis_type='log',
        x_range=[750, 1100000],
        y_range=[0, 0.0275],
        x_axis_label='Dataset Size',
        y_axis_label='Mean Regret (lower is better)',
    )
    num_old_labels = 10
    for total_labels in [20, 50, 100, 200]:
        if total_labels <= num_old_labels:
            continue
        fig = create_color_plot(num_old_labels, total_labels, plot_df, fig)
    fig.outline_line_color = None
    fig.axis.axis_line_width = 5
    fig.axis.axis_label_text_font_size = '30pt'
    fig.axis.major_label_text_font_size = '30pt'
    fig.grid.grid_line_color = None
    fig.axis.minor_tick_in = 0
    fig.axis.minor_tick_out = 0
    fig.legend.location = 'top_right'
    fig.legend.label_text_font_size = '30pt'
    fig.legend.glyph_width = 100
    fig.xaxis[0].formatter = NumeralTickFormatter(format='0a')
    show(fig)

create_color_scaling_plot()

## CIFAR-10, Three Labels Complete Sweep with History

## Sample Regret Plot

### Regret vs. Accuracy

In [ ]:
def agg_cols(df):
    #display(df)
    columns = OrderedDict([
        ('old_labels', df['old_label'].str.cat(sep=',')),
        ('label_mean_regret_scaled', df['label_mean_regret_scaled'].mean()),
        ('mean_regret_scaled', df['mean_regret_scaled'].mean()),
        ('true_positive_init', df['true_positive_init'].mean()),
        ('total_misclass', df['misclassification'].sum()),
        ('total_heuristic', df['heuristic'].sum()),
        ('mean_heuristic', df['heuristic'].mean()),
        ('stdev_heuristic', df['heuristic'].std()),
        ('dev2_heuristic', df['heuristic'].sum() / df['heuristic_rank'].median()),
    ])
    return pd.Series(columns)

def plot_regret_heuristic():
    df = create_image_tribulation('cifar100-threes').groupby(['trial_id', 'new_label']).apply(agg_cols)
    df = df.reset_index()
    #df = df[df['trial_id'] == 'cifar10-l524-b32-e200_cifar10']
    #display(df)
    indicator = df[['new_label', 'old_labels']].apply(
        (lambda row_df: row_df['new_label'] not in row_df['old_labels']),
        axis=1,
    )
    
    df = df[indicator]
    #display(df)
    fig = figure()
    renderer = fig.x(
        x='true_positive_init',
        y='mean_regret_scaled',
        source=ColumnDataSource(df),
    )
    fig.add_tools(HoverTool(renderers=[renderer], tooltips=[
        ('Mean Heuristic', '@mean_heuristic'),
        ('Stdev Heuristic', '@stdev_heuristic'),
        ('Regret', '@label_mean_regret_scaled'),
        ('New Label', '@new_label'),
        ('Old Labels', '@old_labels'),
    ]))
    show(fig)

plot_regret_heuristic()
#create_image_tribulation('cifar10-threes').groupby(['trial_id', 'new_label']).apply(agg_cols)

In [ ]:
def create_regret_accuracy_plot():
    source = create_image_tribulation('cifar10-threes')
    if not isinstance(source, ColumnDataSource):
        source = ColumnDataSource(source)
    fig = figure(
        width=1200, height=1200,
        x_range=[0, 1.1],
        x_axis_label='Classifier Test Accuracy',
        y_range=[0, 1.1],
        y_axis_label='Mean Regret (lower is better)',
    )
    renderer = fig.x(
        x='true_positive_init',
        y='mean_regret_scaled',
        selection_color='red',
        source=source,
        line_width=5,
        size=16,
    )
    fig.add_tools(HoverTool(renderers=[renderer], tooltips=[
        ('trial', '@trial_id'),
        ('new_label', '@new_label'),
        ('accuracy', '@true_positive_init'),
        ('regret', '@mean_regret_scaled'),
    ]))
    
    fig.outline_line_color = None
    fig.axis.axis_line_width = 5
    fig.axis.axis_label_text_font_size = '30pt'
    fig.axis.major_label_text_font_size = '30pt'
    fig.grid.grid_line_color = None
    fig.axis.minor_tick_in = 0
    fig.axis.minor_tick_out = 0
    show(fig)

create_regret_accuracy_plot()


In [ ]:
def create_regret_accuracy_plot():
    source = filter_improvements(create_image_tribulation('cifar10-threes-history-new'))
    display(source.describe())
    if not isinstance(source, ColumnDataSource):
        source = ColumnDataSource(source)
    fig = figure(
        width=1200, height=1200,
        x_range=[0, 1.1],
        x_axis_label='Classifier Test Accuracy',
        y_range=[0, 1.1],
        y_axis_label='Mean Regret (lower is better)',
    )
    renderer = fig.x(
        x='true_positive_init',
        y='mean_regret_scaled',
        selection_color='red',
        source=source,
        line_width=5,
        size=16,
    )
    fig.add_tools(HoverTool(renderers=[renderer], tooltips=[
        ('trial', '@trial_id'),
        ('new_label', '@new_label'),
        ('accuracy', '@true_positive_init'),
        ('regret', '@mean_regret_scaled'),
    ]))
    
    fig.outline_line_color = None
    fig.axis.axis_line_width = 5
    fig.axis.axis_label_text_font_size = '30pt'
    fig.axis.major_label_text_font_size = '30pt'
    fig.grid.grid_line_color = None
    fig.axis.minor_tick_in = 0
    fig.axis.minor_tick_out = 0
    show(fig)

create_regret_accuracy_plot()


In [ ]:
filter_improvements(create_image_tribulation('cifar10-threes-history-new')).corr()

### Plot regret after training

In [ ]:
def plot_accuracy_over_training():
    df = create_image_tribulation('cifar10-threes-history-new')
    df = filter_improvements(df)
    fig = figure(
        width=400, height=400,
        title='Accuracy During Training',
        x_axis_label='Epoch',
        y_axis_label='Accuracy',
        y_range=[0, 1.05],
    )
    for int_labels in df['int_labels'].unique():
        trial_df = df[df['int_labels'] == int_labels]
        trial_df = trial_df[['num_epochs', 'true_positive_init']].drop_duplicates()
        fig.line(
            x='num_epochs',
            y='true_positive_init',
            source=ColumnDataSource(trial_df),
        )
    fig.line(
        x='num_epochs',
        y='true_positive_init',
        color='#CC0000',
        line_width=5,
        source=ColumnDataSource(df[['int_labels', 'num_epochs', 'true_positive_init']].drop_duplicates().pivot_table(
            index='num_epochs',
            values='true_positive_init', 
            aggfunc=np.mean,
        )),
    )
    return fig
    
def plot_regret_over_training():
    df = create_image_tribulation('cifar10-threes-history-new')
    df = filter_improvements(df)
    fig = figure(
        width=400, height=400,
        title='Regret During Training',
        x_axis_label='Epoch',
        y_axis_label='Regret',
        y_range=[0, 1.05],
    )
    for int_labels in df['int_labels'].unique():
        trial_df = df[df['int_labels'] == int_labels]
        trial_df = trial_df[['num_epochs', 'mean_regret_scaled']].drop_duplicates()
        fig.line(
            x='num_epochs',
            y='mean_regret_scaled',
            source=ColumnDataSource(trial_df),
        )
    fig.line(
        x='num_epochs',
        y='mean_regret_scaled',
        color='#CC0000',
        line_width=5,
        source=ColumnDataSource(df[['int_labels', 'num_epochs', 'mean_regret_scaled']].drop_duplicates().pivot_table(
            index='num_epochs',
            values='mean_regret_scaled', 
            aggfunc=np.mean,
        )),
    )
    return fig

show(gridplot([[plot_accuracy_over_training(), plot_regret_over_training()]]))

In [ ]:
def plot_normalized_regret_over_training():
    df = create_image_tribulation('cifar10-threes-history-new')
    df = filter_improvements(df)
    fig = figure(
        width=400, height=400,
        title='Regret During Training',
        x_axis_label='Epoch',
        y_axis_label='Regret',
        #y_range=[0, 1.05],
    )
    for int_labels in df['int_labels'].unique():
        trial_df = df[df['int_labels'] == int_labels]
        trial_df = trial_df[['num_epochs', 'mean_regret_scaled']].drop_duplicates()
        trial_df = trial_df.sort_values('num_epochs')

        #min_epoch = trial_df['num_epochs'].min()
        #initial_regret = trial_df[trial_df['num_epochs'] == min_epoch]['mean_regret_scaled'].iloc[0]
        #trial_df['mean_regret_scaled'] -= initial_regret
        trial_df['regret_diff'] = trial_df['mean_regret_scaled'].diff()
        fig.line(
            x='num_epochs',
            y='regret_diff',
            source=ColumnDataSource(trial_df),
        )
    '''
    fig.line(
        x='num_epochs',
        y='mean_regret_scaled',
        color='#CC0000',
        line_width=5,
        source=ColumnDataSource(df[['int_labels', 'num_epochs', 'mean_regret_scaled']].drop_duplicates().pivot_table(
            index='num_epochs',
            values='mean_regret_scaled', 
            aggfunc=np.mean,
        )),
    )
    '''
    show(fig)

plot_normalized_regret_over_training()

## CIFAR-100 Pilot

### Preview Data

In [ ]:
create_image_tribulation('cifar100-pilot').head()

### Plot Regret

In [ ]:
show(plot_tribulation_regret(create_image_tribulation('cifar100-pilot')))

## CIFAR-100 Orthogonal Exploration

In [ ]:
show(plot_tribulation_regret(create_image_tribulation('cifar100-orthog')))

In [ ]:
def plot_cifar100_orthog():
    tribulation = create_image_tribulation('cifar100-orthog')
    labels = list(range(5, 30, 5))
    epochs = list(range(200, 800, 200))
    # sort the trials
    networks = {}
    for trial_id in tribulation['trial_id'].unique():
        classifier_str = trial_id.split('_')[0]
        path = join_path('cifar100-orthog', classifier_str + '.hdf5')
        nn = NeuralNetwork(path)
        key = (len(nn.int_labels), nn.num_epochs)
        networks.setdefault(key, set()).add(trial_id)


    default_labels = 10
    default_epoch = 200

    label_plots = []
    for num_labels in sorted(labels):
        key = (num_labels, default_epoch)
        plot_df = tribulation[tribulation['trial_id'].apply(
            lambda s: any(s.startswith(prefix) for prefix in networks[key])
        )]
        label_plots.append(plot_tribulation_regret(plot_df))
    show(gridplot([label_plots]))

    epoch_plots = []
    for num_epochs in sorted(epochs):
        key = (default_labels, num_epochs)
        plot_df = tribulation[tribulation['trial_id'].apply(
            lambda s: any(s.startswith(prefix) for prefix in networks[key])
        )]
        epoch_plots.append(plot_tribulation_regret(plot_df))
    show(gridplot([epoch_plots]))

plot_cifar100_orthog()

In [ ]:
create_image_tribulation('cifar100-orthog')